In [2]:
%pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/3e/89/cbca600319463a91ae6a46e537d548900ddf2114df66c902344c4fe6bb4c/pandas-2.1.4-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for numpy<2,>=1.23.2 from https://files.pythonhosted.org/packages/2e/54/218ce51bb571a70975f223671b2a86aa951e83abfd2a416a3d540f35115c/numpy-1.26.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.6 MB/s eta 0:00:00
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 8.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 13.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 13.0 MB/s eta 0:00:0000:0100:01
   ━━

In [ ]:
import pandas as pd

In [4]:

def read_csv_to_dataframe(file_path):
    """
    Liest eine CSV-Datei aus dem angegebenen relativen Pfad ein und gibt einen Pandas DataFrame zurück.
    
    :param file_path: Der relative Pfad zur CSV-Datei.
    :return: Ein Pandas DataFrame mit den Daten aus der CSV-Datei.
    """
    try:
        # Verwende Pandas, um die CSV-Datei in einen DataFrame zu lesen
        dataframe = pd.read_csv(file_path)
        
        # Gib den erstellten DataFrame zurück
        return dataframe
    except FileNotFoundError:
        print(f"Die Datei unter dem Pfad '{file_path}' wurde nicht gefunden.")
        return None
    except Exception as e:
        print(f"Fehler beim Lesen der CSV-Datei: {str(e)}")
        return None

relative_path = "../Ergebnisse/evaluation_iteration1_quantitativ.csv"
data_frame = read_csv_to_dataframe(relative_path)

if data_frame is not None:
    print(data_frame.head())

CSV-Datei erfolgreich eingelesen:
  PROBAND_CODE                    ENTWURF  FRAGE_CODE ITEM_CODE  \
0         P_01  FARBCODIERUNG_MIT_LEGENDE  F_SKALA_01   ITEM_01   
1         P_01  FARBCODIERUNG_MIT_LEGENDE  F_SKALA_02   ITEM_02   
2         P_01  FARBCODIERUNG_MIT_LEGENDE  F_SKALA_03   ITEM_03   
3         P_01  FARBCODIERUNG_MIT_LEGENDE  F_SKALA_04   ITEM_04   
4         P_01  FARBCODIERUNG_MIT_LEGENDE  F_SKALA_05   ITEM_05   

                                          FRAGE_TEXT ITEM_REFERENZ  \
0  Die Darstellung von konkreten Aufrufbeziehunge...   Darstellung   
1  Die Anwendung der Farbcodierung mit Legende zu...     Anwendung   
2  Die Darstellung von konkreten Aufrufbeziehunge...   Darstellung   
3  Die Darstellung von konkreten Aufrufbeziehunge...   Darstellung   
4  Das Gesamtbild erscheint mir durch die Farbcod...    Gesamtbild   

           ITEM_NEG          ITEM_POS          DIMENSION  RESULTAT  
0    unverständlich      verständlich  DURCHSCHAUBARKEIT         3  
1  s

In [ ]:

def read_csv_to_dataframe(file_path):
    """
    Liest eine CSV-Datei aus dem angegebenen relativen Pfad ein und gibt einen Pandas DataFrame zurück.
    
    :param file_path: Der relative Pfad zur CSV-Datei.
    :return: Ein Pandas DataFrame mit den Daten aus der CSV-Datei.
    """
    try:
        # Verwende Pandas, um die CSV-Datei in einen DataFrame zu lesen
        dataframe = pd.read_csv(file_path)
        
        # Gib den erstellten DataFrame zurück
        return dataframe
    except FileNotFoundError:
        print(f"Die Datei unter dem Pfad '{file_path}' wurde nicht gefunden.")
        return None
    except Exception as e:
        print(f"Fehler beim Lesen der CSV-Datei: {str(e)}")
        return None

# Beispielaufruf der Funktion mit einem relativen Pfad
relative_path = "relativer/pfad/zur/deiner_datei.csv"
data_frame = read_csv_to_dataframe(relative_path)

# Wenn die Datei erfolgreich gelesen wurde, kannst du mit dem DataFrame arbeiten
if data_frame is not None:
    print("CSV-Datei erfolgreich eingelesen:")
    print(data_frame.head())